In [2]:
# importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display
import warnings
from contextlib import contextmanager
import joblib


from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate


from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_log_error

# library options
%matplotlib inline
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')

# Regression

## Read in the Train and Test data

In [3]:
X_train = np.array(pd.read_parquet(r'C:\Users\lb199\rwth-aachen.de\Kai Blumberg - Advanced Topics in Marketing Analytics\data\final\X_train.parquet')).astype('float')
X_test = np.array(pd.read_parquet(r'C:\Users\lb199\rwth-aachen.de\Kai Blumberg - Advanced Topics in Marketing Analytics\data\final\X_test.parquet')).astype('float')  

y_test_reg = np.array(pd.read_parquet(r'C:\Users\lb199\rwth-aachen.de\Kai Blumberg - Advanced Topics in Marketing Analytics\data\final\y_test.parquet')['sum_payments_package_key_ltv']).astype('float')
y_train_reg = np.array(pd.read_parquet(r'C:\Users\lb199\rwth-aachen.de\Kai Blumberg - Advanced Topics in Marketing Analytics\data\final\y_train.parquet')['sum_payments_package_key_ltv']).astype('float')

## Train the Model

In [4]:
regressor = LinearRegression()  

# Train the model on training data
regressor.fit(X_train, y_train_reg) 

LinearRegression()

In [5]:
y_pred_r = regressor.predict(X_test)

In [7]:
# Limit predicted values to min=0
y_pred_r=np.clip(y_pred_r, a_min=0,a_max=None)

In [8]:
# Show the Actual and Predicted y
df = pd.DataFrame({'Actual': y_test_reg.flatten(), 'Predicted': y_pred_r.flatten()})
df

,Actual,Predicted
0,29.95,29.122879
1,0.99,0.000000
2,7.94,14.409470
3,2.99,0.559067
4,164.00,100.356369
...,...,...
27943,1.98,3.569717
27944,65.87,94.608978
27945,0.99,8.002441
27946,49.95,67.352997


In [9]:
# Model Accuracy
print("MAE: ", metrics.mean_absolute_error(y_test_reg, y_pred_r))
print("MSE: ", metrics.mean_squared_error(y_test_reg, y_pred_r, squared=True))
print("RMSE: ", metrics.mean_squared_error(y_test_reg, y_pred_r, squared=False))
print("R2: ", metrics.r2_score(y_test_reg, y_pred_r))
print("MSLE: ", metrics.mean_squared_log_error(y_test_reg, y_pred_r))

MAE:  15.787350691930065
MSE:  777.1261716908176
RMSE:  27.876982829761502
R2:  0.5536433447914737
MSLE:  0.904720468974807


## 10-fold Cross-validation

In [10]:
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((y_train_reg, y_test_reg), axis=0)

score = cross_val_score(estimator=regressor, X=inputs, y=targets, cv=10, scoring=metrics.make_scorer(metrics.mean_squared_error))
print("MSE:")
print(score)
print("The mean is " + str(score.mean()) + " with a standard deviation of " + str(score.std()))

MSE:
[756.15736035 763.95929941 749.65064654 735.53136172 806.54080919
 770.92769669 780.20117062 782.52213731 788.26513301 771.547363  ]
The mean is 770.5302977828229 with a standard deviation of 19.39187553592091


In [11]:
score = cross_val_score(estimator=regressor, X=inputs, y=targets, cv=10, scoring=metrics.make_scorer(metrics.mean_absolute_error))
print("MAE:")
print(score)
print("The mean is " + str(score.mean()) + " with a standard deviation of " + str(score.std()))

MAE:
[15.96502936 16.144839   15.97042495 15.85751105 16.41176321 16.00783886
 16.22219991 16.30708196 16.12466591 16.05916529]
The mean is 16.107051950971616 with a standard deviation of 0.1615866641097618


In [12]:
score = cross_val_score(estimator=regressor, X=inputs, y=targets, cv=10, scoring=metrics.make_scorer(metrics.mean_absolute_percentage_error))
print("MAPE:")
print(score)
print("The mean is " + str(score.mean()) + " with a standard deviation of " + str(score.std()))

MAPE:
[1.93768079 1.91371458 1.93691352 1.89160578 1.88797274 1.93799428
 1.94898319 1.96116962 1.90241226 1.91208   ]
The mean is 1.9230526770363099 with a standard deviation of 0.0236807612470667


# Regression without the transactional Data

## Read in the Train and Test data

In [13]:
X_train = np.array(pd.read_parquet(r'C:\Users\lb199\rwth-aachen.de\Kai Blumberg - Advanced Topics in Marketing Analytics\data\final\X_train.parquet').drop(['sum_day_1','sum_day_2','sum_day_3','gradient','clumpiness'], axis=1)).astype('float')
X_test = np.array(pd.read_parquet(r'C:\Users\lb199\rwth-aachen.de\Kai Blumberg - Advanced Topics in Marketing Analytics\data\final\X_test.parquet').drop(['sum_day_1','sum_day_2','sum_day_3','gradient','clumpiness'], axis=1)).astype('float')  

y_test_reg = np.array(pd.read_parquet(r'C:\Users\lb199\rwth-aachen.de\Kai Blumberg - Advanced Topics in Marketing Analytics\data\final\y_test.parquet')['sum_payments_package_key_ltv']).astype('float')
y_train_reg = np.array(pd.read_parquet(r'C:\Users\lb199\rwth-aachen.de\Kai Blumberg - Advanced Topics in Marketing Analytics\data\final\y_train.parquet')['sum_payments_package_key_ltv']).astype('float')

## Train the Model 

In [14]:
regressor = LinearRegression()  

# Train the model on training data
regressor.fit(X_train, y_train_reg) 

LinearRegression()

In [15]:
y_pred_r = regressor.predict(X_test)

In [16]:
# Limit predicted values to min=0
y_pred_r=np.clip(y_pred_r, a_min=0,a_max=None)

In [17]:
# Show the Actual and Predicted y
df = pd.DataFrame({'Actual': y_test_reg.flatten(), 'Predicted': y_pred_r.flatten()})
df

,Actual,Predicted
0,29.95,28.609375
1,0.99,0.000000
2,7.94,14.250000
3,2.99,2.199219
4,164.00,104.445312
...,...,...
27943,1.98,4.226562
27944,65.87,95.296875
27945,0.99,6.035156
27946,49.95,71.753906


In [18]:
# Model Accuracy
print("MAE: ", metrics.mean_absolute_error(y_test_reg, y_pred_r))
print("MSE: ", metrics.mean_squared_error(y_test_reg, y_pred_r, squared=True))
print("RMSE: ", metrics.mean_squared_error(y_test_reg, y_pred_r, squared=False))
print("R2: ", metrics.r2_score(y_test_reg, y_pred_r))
print("MSLE: ", metrics.mean_squared_log_error(y_test_reg, y_pred_r))

MAE:  16.01101219787641
MSE:  791.3349259535627
RMSE:  28.130675888672897
R2:  0.5454822864480133
MSLE:  0.9127165016061188


## 10-fold Cross-validation

In [19]:
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((y_train_reg, y_test_reg), axis=0)

score = cross_val_score(estimator=regressor, X=inputs, y=targets, cv=10, scoring=metrics.make_scorer(metrics.mean_squared_error))
print("MSE:")
print(score)
print("The mean is " + str(score.mean()) + " with a standard deviation of " + str(score.std()))

MSE:
[758.57105129 774.87192844 765.03574028 746.34779136 812.77413889
 787.79886781 794.50596385 796.48496047 803.15110456 785.81243348]
The mean is 782.5353980447601 with a standard deviation of 19.911092880157593


In [20]:
score = cross_val_score(estimator=regressor, X=inputs, y=targets, cv=10, scoring=metrics.make_scorer(metrics.mean_absolute_error))
print("MAE:")
print(score)
print("The mean is " + str(score.mean()) + " with a standard deviation of " + str(score.std()))

MAE:
[16.20271664 16.38109706 16.22461281 16.11966316 16.69130513 16.27306268
 16.47894624 16.55080179 16.36378873 16.3149791 ]
The mean is 16.360097334602706 with a standard deviation of 0.16467862356029347


In [21]:
score = cross_val_score(estimator=regressor, X=inputs, y=targets, cv=10, scoring=metrics.make_scorer(metrics.mean_absolute_percentage_error))
print("MAPE:")
print(score)
print("The mean is " + str(score.mean()) + " with a standard deviation of " + str(score.std()))

MAPE:
[1.93243501 1.90432842 1.93767933 1.89286771 1.88302634 1.92854942
 1.94915174 1.95205274 1.8876703  1.91323407]
The mean is 1.9180995091259323 with a standard deviation of 0.02412708501953769
